In [3]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
import numpy as np
import os

# Example: Load dataset (replace with your dataset path and labels)
def load_data(data_dir):
    images, labels = [], []
    for label in os.listdir(data_dir):
        label_path = os.path.join(data_dir, label)
        for img_file in os.listdir(label_path):
            img_path = os.path.join(label_path, img_file)
            img = tf.keras.preprocessing.image.load_img(img_path, target_size=(64, 64))
            img_array = tf.keras.preprocessing.image.img_to_array(img)
            images.append(img_array)
            labels.append(int(label))
    return np.array(images), np.array(labels)

data_dir = "C:/Users/yaswa/Downloads/archive/Train"
X, y = load_data(data_dir)

# Normalize and split the dataset
X = X / 255.0  # Normalize pixel values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert labels to one-hot encoding
num_classes = len(np.unique(y))
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)


In [5]:
def create_traffic_sign_cnn(input_shape, num_classes):
    model = Sequential([
        # First Convolutional Block
        Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.25),
        
        # Second Convolutional Block
        Conv2D(64, (3, 3), activation='relu'),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.25),
        
        # Third Convolutional Block
        Conv2D(128, (3, 3), activation='relu'),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.25),
        
        # Fully Connected Layers
        Flatten(),
        Dense(256, activation='relu'),
        BatchNormalization(),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])
    return model


In [7]:
model = create_traffic_sign_cnn((64, 64, 3), num_classes)
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [9]:
datagen = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=False,  # Horizontal flip might not make sense for traffic signs
    fill_mode='nearest'
)
datagen.fit(X_train)


In [11]:
batch_size = 32
epochs = 25

history = model.fit(
    datagen.flow(X_train, y_train, batch_size=batch_size),
    validation_data=(X_test, y_test),
    epochs=epochs,
    steps_per_epoch=len(X_train) // batch_size,
    verbose=1
)


Epoch 1/25


980/980 [==============================] - 63s 61ms/step - loss: 1.5389 - accuracy: 0.5717 - val_loss: 0.3098 - val_accuracy: 0.9065
Epoch 2/25
980/980 [==============================] - 59s 61ms/step - loss: 0.4184 - accuracy: 0.8712 - val_loss: 0.0737 - val_accuracy: 0.9805
Epoch 3/25
980/980 [==============================] - 59s 60ms/step - loss: 0.2599 - accuracy: 0.9197 - val_loss: 0.0755 - val_accuracy: 0.9832
Epoch 4/25
980/980 [==============================] - 94s 96ms/step - loss: 0.1902 - accuracy: 0.9395 - val_loss: 0.0274 - val_accuracy: 0.9916
Epoch 5/25
980/980 [==============================] - 59s 61ms/step - loss: 0.1564 - accuracy: 0.9523 - val_loss: 0.0290 - val_accuracy: 0.9909
Epoch 6/25
980/980 [==============================] - 60s 61ms/step - loss: 0.1436 - accuracy: 0.9547 - val_loss: 0.0251 - val_accuracy: 0.9938
Epoch 7/25
980/980 [==============================] - 59s 60ms/step - loss: 0.1226 - accuracy: 0.9623 - val_loss: 0.0107 - val_accurac

In [12]:
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")


Test Accuracy: 99.83%


In [15]:
import pickle
pickle.dump(model,open('model1.pkl','wb'))